In [669]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

class agent:
    def __init__(self, index, race, position):
        self.index = index
        self.race = race # 1 = black, 0 = white
        self.position = position

    def choose_neighborhoob(self, lattice):
        available_neighborhoobs = [self.position, *lattice.vaccant_positions] # k
        probabilities = []
        
        if lattice.choice_functoin == "Schelling":
            for n in available_neighborhoobs:
                # check whether the neighborhoob has at least 50% same race
                neighborhood = lattice.neighborhoods[n]
                agent_in_neighborhood = []
                for p in neighborhood:
                    x, y = to_coordinate(p, lattice.size)
                    agent_in_neighborhood.append(lattice.map[x][y])
                agent_in_neighborhood = [a for a in agent_in_neighborhood if a != -1]
                if len(agent_in_neighborhood) == 0:
                    probabilities.append(0)
                else:                    
                    same_race_proportion = sum([lattice.agents[a].race == self.race for a in agent_in_neighborhood]) / len(agent_in_neighborhood)
                    probabilities.append(int(same_race_proportion >= 0.5))
            if sum(probabilities) == 0:
                lattice.plot()
            probabilities = [p/sum(probabilities) for p in probabilities]

        
        destination = random.choices(available_neighborhoobs, weights=probabilities, k=1)[0]
        self.position = destination
        return destination

class lattice:
    def __init__(self, size, choice_funtion):
        self.size = size
        self.agent_num = int(size**2 * 0.85)
        self.choice_functoin = choice_funtion
        self.t = 0

        # List storing agent object
        self.agents = []
        for agent_index in range(self.agent_num):
            # Decide the agent's race
            race = 0
            x, y =  to_coordinate(agent_index, size)
            if x%2 != y%2:
                race = 1
            
            self.agents.append(agent(agent_index, race, agent_index))
        
        # 2D array recording each position populated by which agent 
        self.map = np.full((self.size, self.size), -1)
        for i in range(self.agent_num):
            x, y =  to_coordinate(i, self.size)
            self.map[x, y] = self.agents[i].index

        # List storing every position's neighborhood
        self.neighborhoods = []
        for p in range(size**2):
            x, y =  to_coordinate(p, size)
            neighbors = []
            for i in range(-2, 3):
                for j in range(-2, 3):  
                    if x+i >= 0 and x+i < size and y+j >= 0 and y+j < size and not (i == 0 and j == 0):
                        neighbor = from_coordinate(x+i, y+j, size)
                        neighbors.append(neighbor)
            self.neighborhoods.append(neighbors)

        # List storing vaccant positions        
        self.vaccant_positions = list(range(self.agent_num, self.size**2))
    
    def one_round(self):
        self.t += 1
        sampled_agent = random.randrange(self.agent_num)
        previous_position = self.agents[sampled_agent].position
        destination = self.agents[sampled_agent].choose_neighborhoob(self)

        if destination != previous_position:
            self.vaccant_positions.remove(destination)
            self.vaccant_positions.append(previous_position)

        x0, y0 = to_coordinate(previous_position, self.size)
        self.map[x0][y0] = -1
        x1, y1 = to_coordinate(destination, self.size)
        self.map[x1][y1] = self.agents[sampled_agent].index        

    def get_dissimilarity(self):
        Nb = sum([a.race for a in self.agents])
        Nw = self.agent_num - Nb

        dissimilarity = 0
        tract_length = 10
        one_side_tract_num = int(self.size/tract_length)
        for tract in range(one_side_tract_num):
            x, y = to_coordinate(tract, one_side_tract_num)
            x *= tract_length
            y *= tract_length

            positions_in_this_tract = []
            for i in range(tract_length):
                for j in range(tract_length):
                    positions_in_this_tract.append((x+i, y+j))

            agents_in_this_tract = [Mylattice.map[p[0]][p[1]] for p in positions_in_this_tract]
            agents_in_this_tract = [a for a in agents_in_this_tract if a != -1]

            Nb_in_this_tract = sum([Mylattice.agents[a].race for a in agents_in_this_tract])
            Nw_in_this_tract = len(agents_in_this_tract) - Nb_in_this_tract

            dissimilarity += abs(Nb_in_this_tract/Nb - Nw_in_this_tract/Nw)

            return (dissimilarity / 2)

    def plot(self, tick_num=10, show_grid=False):
        map_race = np.full((self.size, self.size), -1)
        for i in range(self.agent_num):
            x, y =  to_coordinate(self.agents[i].position, self.size)
            map_race[x, y] = self.agents[i].race
        
        cmap = colors.ListedColormap(['white', 'navajowhite', 'midnightblue'])
        plt.figure(figsize=(8, 8))
        plt.pcolormesh(map_race, cmap=cmap, alpha = 0.8, vmin=-1, vmax=1) 
        ax = plt.gca()
        ax.set_xticks(np.arange(0, self.size+1, max(1, int(self.size/tick_num))))
        ax.set_yticks(np.arange(0, self.size+1, max(1, int(self.size/tick_num))))
        if show_grid:
            ax.grid(c='black', linestyle='-', linewidth=.75, alpha=.75)
        plt.show()

def to_coordinate(i, col):
    x, y = int(i/col), i%col
    return (x, y)

def from_coordinate(x, y, col):
    return x*col+y

In [851]:
Mylattice = lattice(500, 'Schelling')
log = []
round_num = 10**6
for round in range(round_num):
    Mylattice.one_round()
    if round % 10 == 0: 
        log.append(Mylattice.get_dissimilarity())
    if round % 10**5 == 0:
        print(round, end=" ")

fig, ax = plt.subplots(1, 1, figsize=(9, 6))

ax.set_xlim([0, 10**6])
ax.set_ylim([0, 0.4])
plt.plot(log, label="Schelling Function")

ax.set_xlabel("ticks (in 10000)", fontsize=12)
ax.set_ylabel("Dissimilarity", rotation=90, y=0.5, fontsize=12)
ax.set_yticks(np.arange(0, 0.41, step=0.04))
ax.set_xticks(np.arange(0, 10**6+1, step=10**5))
ax.xaxis.set_major_formatter(lambda x, pos: f'{x/10000:.2f}'.rstrip('0').rstrip('.'))
ax.yaxis.set_major_formatter(lambda y, pos: f'{y:.2f}'.rstrip('0').rstrip('.'))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.set_facecolor('white')
plt.legend()
plt.savefig('FIG3.jpg')
plt.show()

0 100000 200000 300000 400000 500000 600000 700000 800000 900000 